In [1]:
API_KEY = 'xxx'
PROJECT_ID = 'xxx'

In [4]:
import os
import json
from PIL import Image

# category mapping (from old ontology to new ontology)
category_mapping = {xxx:xxx}

# get image size
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size  # (width, height)

# function to convert yolo format to labelbox format
def yolo_to_labelbox(yolo_label, img_width, img_height):
    parts = yolo_label.split()
    category = int(parts[0])
    center_x_ratio = float(parts[1])
    center_y_ratio = float(parts[2])
    width_ratio = float(parts[3])
    height_ratio = float(parts[4])
    
    # convert to pixel coordinate
    center_x = center_x_ratio * img_width
    center_y = center_y_ratio * img_height
    width = width_ratio * img_width
    height = height_ratio * img_height
    
    # calculate the coordinate of top-life and bottom-right point 
    top = center_y - (height / 2)
    left = center_x - (width / 2)
    bottom = center_y + (height / 2)
    right = center_x + (width / 2)
    
    # build the annotation object of labelbox
    labelbox_annotation = {
        "name": "Vehicle Class Ontology",
        "value": {
            "start": {"x": left, "y": top},
            "end": {"x": right, "y": bottom}
        },
        "classifications": [{
            "name": "Vehicle type",
            "value": {
                "answer": {
                    "name": "0"
                }
            }
        }]
    }
    
    return labelbox_annotation

In [5]:
# convert all txt file 
def process_folder(label_folder, image_folder, output_folder):
    result = {}
    for file in os.listdir(label_folder):
        if file.endswith(".txt"):
            image_path = os.path.join(image_folder, file.replace('.txt', '.jpg'))  # 假设图片扩展名为.jpg
            
            if not os.path.exists(image_path):
                print(f"Warning: Image {image_path} not found. Skipping corresponding label {file}.")
                continue
            
            with open(os.path.join(label_folder, file), 'r') as f:
                labels = f.readlines()
            

            img_width, img_height = get_image_size(image_path)
       
            converted_labels = [yolo_to_labelbox(label.strip(), img_width, img_height) for label in labels]
         
            result[file.replace('.txt', '')] = converted_labels
    
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            output_file_path = os.path.join(output_folder, file.replace('.txt', '.json'))
            with open(output_file_path, 'w') as f:
                json.dump(converted_labels, f, indent=4)

process_folder(r'label folder', r'image folder', r'converted label folder')

In [6]:
OUTPUT_DIRECTORY_PATH = r'converted label folder'

In [7]:
import os
import requests

# If the graph in the project doesn't have global key, you must get the mapping between external_id and ids to upload them(by id)

file_names = [f[:-5] for f in os.listdir(OUTPUT_DIRECTORY_PATH) if f.endswith('.json')]
external_ids = [f"{name}.jpg" for name in file_names]

headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

query = """
query GetDataRowID($projectId: ID!, $externalId: String!) {
  project(where: {id: $projectId}) {
    dataRows(where: {externalId: $externalId}) {
      id
    }
  }
}
"""
data_row_ids = []
id_mapping = {}

for external_id in external_ids:
    variables = {
        'projectId': PROJECT_ID,
        'externalId': external_id
    }

    response = requests.post(
        'https://api.labelbox.com/graphql',
        headers=headers,
        json={'query': query, 'variables': variables}
    )

    data = response.json()
    data_row_id = data['data']['project']['dataRows'][0]['id']
    data_row_ids.append(data_row_id)
    id_mapping[external_id] = data_row_id

print(id_mapping)

{'1882--1-1-2_exp394.jpg': 'cln6nuaco0gh50774dc5i3o8g', '1882--1-1-2_exp395.jpg': 'cln6nuaco0gh10774e1xaeup6', '1882--1-1-2_exp396.jpg': 'cln6nuaco0ggx07743t8baj8v', '1882--1-1-2_exp397.jpg': 'cln6nuaco0ggt0774g3nbg6fx', '1882--1-1-2_exp398.jpg': 'cln6nuaco0ggp077464jz8pdv', '1883--1-0-2_exp323.jpg': 'cln6nuacq0gl10774bjaq9r2z', '1883--1-0-2_exp324.jpg': 'cln6nuacq0gkx077409530hpv', '1883--1-0-2_exp326.jpg': 'cln6nuacq0gkt0774ftu34s6r', '1883--1-0-2_exp327.jpg': 'cln6nuacq0gkp0774dp9y53z0', '1883--1-0-2_exp328.jpg': 'cln6nuacq0gkl0774407x7tbg', '1884--1-0-2_exp610.jpg': 'cln6nuack0g9d0774gu6517l4', '1884--1-0-2_exp611.jpg': 'cln6nuack0g99077401jx6z1u', '1884--1-0-2_exp612.jpg': 'cln6nuack0g950774au52eb2y', '1884--1-0-2_exp613.jpg': 'cln6nuack0g9107745sxn8pnf', '1884--1-0-2_exp618.jpg': 'cln6nuack0g8x0774cki549zl', '1884--1-1-2_exp353.jpg': 'cln6nuacq0gjx0774cv558khm', '1884--1-1-2_exp354.jpg': 'cln6nuacq0gjt0774dpmx46bg', '1884--1-1-2_exp355.jpg': 'cln6nuacq0gjp0774exq8db0v', '1884--1-

In [8]:
import os
import json
import uuid
import labelbox as lb
import labelbox.data.annotation_types as lb_types

client = lb.Client(API_KEY)
project = client.get_project(PROJECT_ID)

def convert_to_labelbox_format(label_folder):
    all_annotations = []
    for file in os.listdir(label_folder):
        if file.endswith(".json"):
            with open(os.path.join(label_folder, file), 'r') as f:
                annotations_data = json.load(f)

            annotations = []
            for annotation in annotations_data:
                bbox = lb_types.ObjectAnnotation(
                    name=annotation['name'],
                    value=lb_types.Rectangle(
                        start=lb_types.Point(x=annotation['value']['start']['x'], y=annotation['value']['start']['y']),
                        end=lb_types.Point(x=annotation['value']['end']['x'], y=annotation['value']['end']['y'])
                    )
                )
                annotations.append(bbox)
                
            external_id = file.replace('.json', '.jpg')
            uid_value = id_mapping.get(external_id)

            image_data = lb_types.ImageData(uid= uid_value)
            
            label_data = lb_types.Label(data=image_data, annotations=annotations)
            
            all_annotations.append(label_data)  
    
    return all_annotations

labels = convert_to_labelbox_format(r'converted label folder')

print(labels)

[Label(uid=None, data=ImageData(im_bytes=None,file_path=None,url=None,arr=None), annotations=[ObjectAnnotation(confidence=None, name='Vehicle Class Ontology', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=15.999968000000003, y=238.00015199999999), end=Point(extra={}, x=34.999968, y=258.000168)), classifications=[]), ObjectAnnotation(confidence=None, name='Vehicle Class Ontology', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=0.0, y=236.00016000000002), end=Point(extra={}, x=18.0, y=272.00016000000005)), classifications=[]), ObjectAnnotation(confidence=None, name='Vehicle Class Ontology', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=53.999968, y=243.99983999999998), end=Point(extra={}, x=72.999968, y=261.99983999999995)), classifications=[]), ObjectAnnotation(confidence=None, name='Vehicle Class Ontology', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Po

In [10]:
upload_job_label_import = lb.LabelImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="label_import_job_success5",
    labels=labels
)